In [6]:
import logging
import requests
import datetime as dt
from dateutil.parser import parse

module_logger = logging.getLogger('cta_train_collector')

class CTA:
    def __init__(self, cta_api_key):
        self.logger = logging.getLogger('cta_train_collector.cta_api.CTA')
        self.logger.info('creating an instance of CTA')
        self.api_key = cta_api_key

    def parse_train_response(self, response):
        trains = []
        try:
            timestamp = response['ctatt']['tmst']
            time = parse(timestamp)
            for route in response['ctatt']['route']:
                name = route['@name']
                if 'train' in route.keys():
                    if(type(route['train']) is dict):
                        route['train'] = [route['train']] # convert dict to singleton list
                    for train in route['train']:
                        train['line'] = name
                        train['tmst'] = timestamp
                        train['time'] = time
                        trains.append(train)
        except ValueError as e:
            self.logger.error(e)
        return trains

    def get_train_data(self):
        logging.basicConfig(filename="logfile")
        response = {}
        url = f'http://lapi.transitchicago.com/api/1.0/ttpositions.aspx?key={self.api_key}&rt=red,blue,brn,org,p,pink,y&outputType=JSON'
        try:
            response = requests.get(url).json()
        except requests.exceptions.RequestException as e:  
            logging.exception(e)
        except requests.exceptions.HTTPError as e:
            logging.exception(e)
        return response


In [7]:
import logging
# from cta_api import CTA
from config import cta_api_key
import datetime as dt
import pymongo
from time import sleep


logger = logging.getLogger('cta_train_collector')
fh = logging.FileHandler('cta_train_collector.log')
fh.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
logger.addHandler(fh)

conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)
db = client.cta_db

cta = CTA(cta_api_key)

In [8]:
response = cta.get_train_data()
trains = cta.parse_train_response(response)

2019-03-25 16:06:52


In [9]:
trains

[{'rn': '810',
  'destSt': '30173',
  'destNm': 'Howard',
  'trDr': '1',
  'nextStaId': '41230',
  'nextStpId': '30237',
  'nextStaNm': '47th (Red)',
  'prdt': '2019-03-25T16:06:28',
  'arrT': '2019-03-25T16:08:28',
  'isApp': '0',
  'isDly': '0',
  'flags': None,
  'lat': '41.79542',
  'lon': '-87.63117',
  'heading': '358',
  'line': 'red',
  'tmst': '2019-03-25T16:06:52',
  'time': datetime.datetime(2019, 3, 25, 16, 6, 52)},
 {'rn': '811',
  'destSt': '30173',
  'destNm': 'Howard',
  'trDr': '1',
  'nextStaId': '40940',
  'nextStpId': '30183',
  'nextStaNm': 'Halsted',
  'prdt': '2019-03-25T16:06:23',
  'arrT': '2019-03-25T16:08:23',
  'isApp': '0',
  'isDly': '0',
  'flags': None,
  'lat': '41.77908',
  'lon': '-87.65681',
  'heading': '89',
  'line': 'red',
  'tmst': '2019-03-25T16:06:52',
  'time': datetime.datetime(2019, 3, 25, 16, 6, 52)},
 {'rn': '815',
  'destSt': '30057',
  'destNm': 'Ashland/63rd',
  'trDr': '5',
  'nextStaId': '41690',
  'nextStpId': '30382',
  'nextStaNm'

In [10]:
    timestamp = dt.datetime.now()
    response['timestamp'] = timestamp
    db.cta_raw_responses.insert_one(response)
    trains = cta.parse_train_response(response)
    db.cta_trains.insert_many(trains)


2019-03-25 16:06:52


In [ ]:
trains

In [13]:
db.cta_trains.aggregate([
    { "$addFields" : {
        "time": { '$dateFromString': {'dateString': '$tmst'} }
    }},
    { "$out": "cta_trains"}
])